In [6]:
# Cell 1: Read CSV (safe reading, no inference on dtypes)
from pathlib import Path
import pandas as pd
import logging

logging.basicConfig(level=logging.INFO, format="%(levelname)s: %(message)s")

# --- CONFIG: point to your CSV file here ---
csv_path = Path(r"C:\Users\ishaa\OneDrive\Desktop\Features-main\Features-main\data\output\vehicle_alerts_decoded_json.csv")

if not csv_path.exists():
    raise FileNotFoundError(f"CSV not found: {csv_path}")

# Read all columns as strings to avoid accidental type coercion
df = pd.read_csv(csv_path, dtype=str, keep_default_na=False, na_values=[""])

logging.info(f"Read CSV: {csv_path}  (rows={len(df):,}, cols={len(df.columns):,})")
display(df.head(3))


INFO: Read CSV: C:\Users\ishaa\OneDrive\Desktop\Features-main\Features-main\data\output\vehicle_alerts_decoded_json.csv  (rows=99, cols=15)


,alert_id,vehicle_id,module,start_ts,end_ts,duration_s,n_points,peak_composite,mean_composite,severity,severity_label,top_features_b64,date,row_hash,top_features_b64_decoded_json
0,956ce7b4a44145d9,sim001,battery,2024-07-05T07:22:11+00:00,2024-07-05T07:48:51+00:00,1600,1409,0.750220971406238,0.5006715535033771,1,anomaly,W3siZmVhdHVyZSI6Imh2X2JhdHRlcnlfcGFja19jdXJyZW...,2024-07-05,55946eee8a0573b42d51b2f06a7049600aceae5bf7d992...,"[{""feature"":""hv_battery_pack_current"",""contrib..."
1,a81c57582cfe4067,sim001,battery,2024-07-05T07:50:48+00:00,2024-07-05T07:50:48+00:00,0,1,0.4853907478407508,0.4853907478407508,1,anomaly,W3siZmVhdHVyZSI6ImJhdHRlcnlfdm9sdGFnZV9lY3VfN2...,2024-07-05,ff4022f76cd1346489b0e657c7d6ae79919bbffba8eb79...,"[{""feature"":""battery_voltage_ecu_7ee"",""contrib..."
2,f7de643eeb7a405c,sim001,battery,2024-07-05T07:52:55+00:00,2024-07-05T07:54:51+00:00,116,74,0.5092871064786861,0.4929241629486659,1,anomaly,W3siZmVhdHVyZSI6ImJhdHRlcnlfdm9sdGFnZV9lY3VfN2...,2024-07-05,4d4086acf9e59bcae0891a30c455a915e9c5884244a108...,"[{""feature"":""battery_voltage_ecu_7ee"",""contrib..."


In [7]:
# Cell 2: Ensure uniqueness of row-level (original) rows by row_hash
if 'row_hash' not in df.columns:
    raise KeyError("Expected column 'row_hash' not found in CSV")

# Check uniqueness
n_total = len(df)
n_unique = df['row_hash'].nunique()
if n_total != n_unique:
    # Print a small diagnostic and stop (per your instruction: no point proceeding)
    dup_counts = df['row_hash'].value_counts()
    duplicates = dup_counts[dup_counts > 1]
    logging.error("row_hash is NOT unique across rows — aborting further expansion.")
    logging.error(f"Total rows: {n_total}, Unique row_hash: {n_unique}, Duplicates: {len(duplicates)}")
    display(duplicates.head(10))
    raise SystemExit("row_hash uniqueness check failed. Resolve duplicate row_hash entries before proceeding.")
else:
    logging.info("All rows unique by row_hash — OK to proceed.")


INFO: All rows unique by row_hash — OK to proceed.


In [8]:
# Cell 3 (UPDATED): Expand top_features_b64_decoded_json into multiple rows,
# while keeping ALL original columns exactly duplicated for each expanded row.

import json
import re
from typing import List, Dict, Any

col = 'top_features_b64_decoded_json'
if col not in df.columns:
    raise KeyError(f"Expected column '{col}' not found in CSV")

def safe_parse_features(s: str) -> List[Dict[str, Any]]:
    """
    Robust parser for lists of {feature, contribution} dictionaries inside CSV.
    Handles:
      - Proper JSON
      - CSV escaped JSON with doubled quotes
      - Bracket substring extraction
      - Regex fallback extraction
    """
    if s is None:
        return []
    s = s.strip()
    if s == "" or s.lower() in ("nan", "none"):
        return []

    # Attempt 1: direct JSON
    try:
        parsed = json.loads(s)
        if isinstance(parsed, list):
            return parsed
    except Exception:
        pass

    # Attempt 2: replace doubled double-quotes and retry
    try:
        s2 = s.replace('""', '"')
        parsed = json.loads(s2)
        if isinstance(parsed, list):
            return parsed
    except Exception:
        pass

    # Attempt 3: extract the substring between '[' ... ']'
    try:
        start = s.find('[')
        end = s.rfind(']')
        if start != -1 and end != -1:
            substring = s[start:end+1].replace('""', '"')
            parsed = json.loads(substring)
            if isinstance(parsed, list):
                return parsed
    except Exception:
        pass

    # Attempt 4: regex fallback
    items = []
    for match in re.finditer(r'\{[^}]*\}', s):
        chunk = match.group(0)
        # Try safe JSON load on each object
        try:
            cand = json.loads(chunk.replace('""', '"'))
            if isinstance(cand, dict):
                items.append(cand)
                continue
        except:
            pass
        # Manual extract feature + contribution
        f_m = re.search(r'"?feature"?\s*:\s*"?([^",}]+)"?', chunk)
        c_m = re.search(r'"?contribution"?\s*:\s*("?[-0-9.eE+]+"?)', chunk)
        if f_m:
            feature = f_m.group(1).strip()
            contrib_raw = c_m.group(1).strip() if c_m else ""
            items.append({"feature": feature, "contribution": contrib_raw})

    return items

def normalize_contribution(raw) -> float:
    """
    Convert contribution (possibly messy string) into float.
    Strips leading colons, stray quotes, whitespace.
    """
    if raw is None:
        raise ValueError("Contribution is None")

    if isinstance(raw, (int, float)):
        return float(raw)

    s = str(raw).strip()
    s = s.lstrip(':').strip().strip('"').strip("'")

    # Keep only the first float-like pattern
    m = re.search(r'[-+]?\d+(?:\.\d+)?(?:[eE][-+]?\d+)?', s)
    if not m:
        raise ValueError(f"Cannot parse contribution float from: {raw!r}")

    return float(m.group(0))


# ---- EXPANSION WITH ALL ORIGINAL COLUMNS ----

expanded_rows = []

original_cols = list(df.columns)   # keep full schema

for _, row in df.iterrows():
    rh = row['row_hash']
    raw = row.get(col, "")

    parsed_list = safe_parse_features(raw)
    if not parsed_list:
        logging.warning(f"No parsed features for row_hash={rh!s}; skipping expansion.")
        continue

    for item in parsed_list:
        feat = item.get("feature")
        contrib_raw = item.get("contribution")

        contrib = normalize_contribution(contrib_raw)

        # Copy ALL original columns for the expanded row
        new_row = {colname: row[colname] for colname in original_cols}

        # Add new feature-specific fields
        new_row["feature"] = feat
        new_row["contribution"] = contrib

        expanded_rows.append(new_row)

expanded_df = pd.DataFrame(expanded_rows)

logging.info(f"Expanded rows: {len(expanded_df):,}")
display(expanded_df.head(10))


INFO: Expanded rows: 297


,alert_id,vehicle_id,module,start_ts,end_ts,duration_s,n_points,peak_composite,mean_composite,severity,severity_label,top_features_b64,date,row_hash,top_features_b64_decoded_json,feature,contribution
0,956ce7b4a44145d9,sim001,battery,2024-07-05T07:22:11+00:00,2024-07-05T07:48:51+00:00,1600,1409,0.750220971406238,0.5006715535033771,1,anomaly,W3siZmVhdHVyZSI6Imh2X2JhdHRlcnlfcGFja19jdXJyZW...,2024-07-05,55946eee8a0573b42d51b2f06a7049600aceae5bf7d992...,"[{""feature"":""hv_battery_pack_current"",""contrib...",hv_battery_pack_current,0.814362
1,956ce7b4a44145d9,sim001,battery,2024-07-05T07:22:11+00:00,2024-07-05T07:48:51+00:00,1600,1409,0.750220971406238,0.5006715535033771,1,anomaly,W3siZmVhdHVyZSI6Imh2X2JhdHRlcnlfcGFja19jdXJyZW...,2024-07-05,55946eee8a0573b42d51b2f06a7049600aceae5bf7d992...,"[{""feature"":""hv_battery_pack_current"",""contrib...",energy_consumption_per_km_wh_per_km_per_inr_pe...,0.618046
2,956ce7b4a44145d9,sim001,battery,2024-07-05T07:22:11+00:00,2024-07-05T07:48:51+00:00,1600,1409,0.750220971406238,0.5006715535033771,1,anomaly,W3siZmVhdHVyZSI6Imh2X2JhdHRlcnlfcGFja19jdXJyZW...,2024-07-05,55946eee8a0573b42d51b2f06a7049600aceae5bf7d992...,"[{""feature"":""hv_battery_pack_current"",""contrib...",battery_voltage_ecu_7ee,0.500382
3,a81c57582cfe4067,sim001,battery,2024-07-05T07:50:48+00:00,2024-07-05T07:50:48+00:00,0,1,0.4853907478407508,0.4853907478407508,1,anomaly,W3siZmVhdHVyZSI6ImJhdHRlcnlfdm9sdGFnZV9lY3VfN2...,2024-07-05,ff4022f76cd1346489b0e657c7d6ae79919bbffba8eb79...,"[{""feature"":""battery_voltage_ecu_7ee"",""contrib...",battery_voltage_ecu_7ee,0.061575
4,a81c57582cfe4067,sim001,battery,2024-07-05T07:50:48+00:00,2024-07-05T07:50:48+00:00,0,1,0.4853907478407508,0.4853907478407508,1,anomaly,W3siZmVhdHVyZSI6ImJhdHRlcnlfdm9sdGFnZV9lY3VfN2...,2024-07-05,ff4022f76cd1346489b0e657c7d6ae79919bbffba8eb79...,"[{""feature"":""battery_voltage_ecu_7ee"",""contrib...",fuel_consumption_km_per_l_or_l_per_100_km,0.050318
5,a81c57582cfe4067,sim001,battery,2024-07-05T07:50:48+00:00,2024-07-05T07:50:48+00:00,0,1,0.4853907478407508,0.4853907478407508,1,anomaly,W3siZmVhdHVyZSI6ImJhdHRlcnlfdm9sdGFnZV9lY3VfN2...,2024-07-05,ff4022f76cd1346489b0e657c7d6ae79919bbffba8eb79...,"[{""feature"":""battery_voltage_ecu_7ee"",""contrib...",boost_commanded_per_measured,0.040213
6,f7de643eeb7a405c,sim001,battery,2024-07-05T07:52:55+00:00,2024-07-05T07:54:51+00:00,116,74,0.5092871064786861,0.4929241629486659,1,anomaly,W3siZmVhdHVyZSI6ImJhdHRlcnlfdm9sdGFnZV9lY3VfN2...,2024-07-05,4d4086acf9e59bcae0891a30c455a915e9c5884244a108...,"[{""feature"":""battery_voltage_ecu_7ee"",""contrib...",battery_voltage_ecu_7ee,0.147368
7,f7de643eeb7a405c,sim001,battery,2024-07-05T07:52:55+00:00,2024-07-05T07:54:51+00:00,116,74,0.5092871064786861,0.4929241629486659,1,anomaly,W3siZmVhdHVyZSI6ImJhdHRlcnlfdm9sdGFnZV9lY3VfN2...,2024-07-05,4d4086acf9e59bcae0891a30c455a915e9c5884244a108...,"[{""feature"":""battery_voltage_ecu_7ee"",""contrib...",alternator_voltage_output,0.140254
8,f7de643eeb7a405c,sim001,battery,2024-07-05T07:52:55+00:00,2024-07-05T07:54:51+00:00,116,74,0.5092871064786861,0.4929241629486659,1,anomaly,W3siZmVhdHVyZSI6ImJhdHRlcnlfdm9sdGFnZV9lY3VfN2...,2024-07-05,4d4086acf9e59bcae0891a30c455a915e9c5884244a108...,"[{""feature"":""battery_voltage_ecu_7ee"",""contrib...",regenerative_energy_recovered,0.073656
9,1957d31f1867429d,sim001,battery,2024-07-05T07:58:02+00:00,2024-07-05T07:58:02+00:00,0,1,0.6411275466487082,0.6411275466487082,2,warning,W3siZmVhdHVyZSI6ImJhcm9tZXRlcl9hbmRyb2lkX2Rldm...,2024-07-05,0c1614b6a21b65e94bda6c66d3c680cadaa6a63a16cb29...,"[{""feature"":""barometer_android_device_mb"",""con...",barometer_android_device_mb,0.297476


In [9]:
# Cell 4: Verification
from collections import Counter
import numpy as np

# 1) check count per row_hash
counts = expanded_df['row_hash'].value_counts()
n_bad_counts = (counts != 3).sum()
if n_bad_counts != 0:
    logging.error(f"{n_bad_counts} row_hash values do NOT appear exactly 3 times.")
    display(counts[counts != 3].head(20))
else:
    logging.info("All row_hash values appear exactly 3 times. (OK)")

# 2) check feature is string (non-empty)
bad_feature_mask = expanded_df['feature'].isnull() | (expanded_df['feature'].astype(str).str.strip() == "")
if bad_feature_mask.any():
    logging.error(f"Found {bad_feature_mask.sum()} rows with missing/empty feature.")
    display(expanded_df[bad_feature_mask].head(10))
else:
    logging.info("All feature values are non-empty strings. (OK)")

# 3) check contribution is float (and finite, >=0 typically)
if not np.issubdtype(expanded_df['contribution'].dtype, np.floating):
    # attempt to coerce if needed
    try:
        expanded_df['contribution'] = expanded_df['contribution'].astype(float)
    except Exception as e:
        logging.error("Contribution column is not float and could not be coerced.")
        raise

nan_or_inf = ~np.isfinite(expanded_df['contribution'])
if nan_or_inf.any():
    logging.error(f"Found {nan_or_inf.sum()} contribution values that are NaN or infinite.")
    display(expanded_df[nan_or_inf].head(10))
else:
    logging.info("All contribution values are finite floats. (OK)")

# show a concise verification summary
summary = {
    "original_rows": len(df),
    "expanded_rows": len(expanded_df),
    "unique_row_hashes_in_expanded": expanded_df['row_hash'].nunique(),
    "rows_per_hash_expected": 3,
    "any_non3_counts": int(n_bad_counts > 0)
}
display(summary)
# show a sample tail/head
display(expanded_df.sample(min(10, len(expanded_df)), random_state=1))


INFO: All row_hash values appear exactly 3 times. (OK)
INFO: All feature values are non-empty strings. (OK)
INFO: All contribution values are finite floats. (OK)


{'original_rows': 99,
 'expanded_rows': 297,
 'unique_row_hashes_in_expanded': 99,
 'rows_per_hash_expected': 3,
 'any_non3_counts': 0}

,alert_id,vehicle_id,module,start_ts,end_ts,duration_s,n_points,peak_composite,mean_composite,severity,severity_label,top_features_b64,date,row_hash,top_features_b64_decoded_json,feature,contribution
138,e1e8add558534e5a,sim001,engine,2024-07-05T07:22:10+00:00,2024-07-05T07:37:59+00:00,949,913,0.9112692701799002,0.7147054773287858,3,critical,W3siZmVhdHVyZSI6ImFpcl9mdWVsX3JhdGlvX2NvbW1hbm...,2024-07-05,f160349ff244b6951e9cafda072cfec798a92b4c5691bf...,"[{""feature"":""air_fuel_ratio_commanded_1"",""cont...",air_fuel_ratio_commanded_1,3.938247
233,c36d32aedf424871,sim001,transmission,2024-07-05T12:14:29+00:00,2024-07-05T12:20:47+00:00,378,102,0.622146504679047,0.5303862898746717,1,anomaly,W3siZmVhdHVyZSI6ImFpcl9mdWVsX3JhdGlvX21lYXN1cm...,2024-07-05,946b02a29d844d5db0bfd264372405fbcfa27268e162e3...,"[{""feature"":""air_fuel_ratio_measured"",""contrib...",driver_demand_engine_pct_torque,0.094367
51,407b77efd8dd42b1,sim001,battery,2024-07-05T10:13:36+00:00,2024-07-05T10:16:34+00:00,178,105,0.5025858031274506,0.491585931893237,1,anomaly,W3siZmVhdHVyZSI6ImJhdHRlcnlfdm9sdGFnZV9lY3VfN2...,2024-07-05,9f4e4dd278c5edc906397474d68d0df99d4601e22fe2f2...,"[{""feature"":""battery_voltage_ecu_7ee"",""contrib...",battery_voltage_ecu_7ee,0.051406
290,a1a09468d9fa44c5,sim001,transmission,2024-07-05T14:35:06+00:00,2024-07-05T16:07:38+00:00,5552,2932,0.7685518152374289,0.5480112872950265,1,anomaly,W3siZmVhdHVyZSI6ImFpcl9mdWVsX3JhdGlvX2NvbW1hbm...,2024-07-05,093eace6d608c085b981d2b76ff9eac961a0f3f4cdae53...,"[{""feature"":""air_fuel_ratio_commanded"",""contri...",accelerator_pedal_position_d_per_e_per_f_pct,0.457616
242,a78b95f8365f40df,sim001,transmission,2024-07-05T13:08:24+00:00,2024-07-05T13:24:26+00:00,962,216,0.690337233076285,0.5341197570286078,1,anomaly,W3siZmVhdHVyZSI6ImVncl9jb21tYW5kZWRfcGN0IiwiY2...,2024-07-05,3c96faf9b3ae70ce4e7f5aa0a1b22b9832b13561096e94...,"[{""feature"":""egr_commanded_pct"",""contribution""...",engine_reference_torque_nm,0.182854
244,e33d5889f5ba45fe,sim001,transmission,2024-07-05T13:25:35+00:00,2024-07-05T13:27:03+00:00,88,31,0.6579266803054984,0.5432991000050394,1,anomaly,W3siZmVhdHVyZSI6ImFpcl9mdWVsX3JhdGlvX2NvbW1hbm...,2024-07-05,51d299541d857b383abc12e28a8f09b6ae4b8c222d7d91...,"[{""feature"":""air_fuel_ratio_commanded"",""contri...",air_fuel_ratio_measured,0.251678
272,2f88d058f2cc44fb,sim001,transmission,2024-07-05T14:08:21+00:00,2024-07-05T14:15:58+00:00,457,125,0.666860661223019,0.5389387656724642,1,anomaly,W3siZmVhdHVyZSI6ImFpcl9mdWVsX3JhdGlvX21lYXN1cm...,2024-07-05,b79fdb79241723761c139056600adbf5bbcdaafe95efbc...,"[{""feature"":""air_fuel_ratio_measured"",""contrib...",engine_reference_torque_nm,0.121650
62,eef9cabf6c464613,sim001,battery,2024-07-05T10:45:27+00:00,2024-07-05T10:47:24+00:00,117,99,0.4973232984164107,0.48930934545663307,1,anomaly,W3siZmVhdHVyZSI6ImNvX2VtaXNzaW9uc19pbnN0YW50X3...,2024-07-05,ffcb2ace0f29bac37b63c090ce74b9d86e232bcb3e1c5f...,"[{""feature"":""co_emissions_instant_per_avg"",""co...",hv_battery_pack_current,0.062076
27,b32271bb0b384a5b,sim001,battery,2024-07-05T09:13:10+00:00,2024-07-05T09:14:10+00:00,60,5,0.4886266544580837,0.4859949063678176,1,anomaly,W3siZmVhdHVyZSI6ImFsdGVybmF0b3Jfdm9sdGFnZV9vdX...,2024-07-05,ac73334d657b72efb3cc752649cec246f6ed8dbf58b1bd...,"[{""feature"":""alternator_voltage_output"",""contr...",alternator_voltage_output,0.064607
122,66f0b58139eb4605,sim001,battery,2024-07-05T14:06:48+00:00,2024-07-05T14:07:34+00:00,46,3,0.4924713171196149,0.49021784723265593,1,anomaly,W3siZmVhdHVyZSI6ImJhcm9tZXRlcl9hbmRyb2lkX2Rldm...,2024-07-05,2a6d5a7c7143dbb6f5a920da260946301b4f6a184d93d8...,"[{""feature"":""barometer_android_device_mb"",""con...",co_emissions_instant_per_avg,0.130932


In [10]:
# Cell 5: Write the final CSV to the same directory with "_extended" appended
out_dir = csv_path.parent
in_name = csv_path.stem  # without .csv
out_name = f"{in_name}_extended.csv"
out_path = out_dir / out_name

# It's safe to overwrite by default; if you want, add a versioning/timestamp check
expanded_df.to_csv(out_path, index=False)

logging.info(f"Wrote expanded CSV to: {out_path}  (rows={len(expanded_df):,})")
display(out_path)


INFO: Wrote expanded CSV to: C:\Users\ishaa\OneDrive\Desktop\Features-main\Features-main\data\output\vehicle_alerts_decoded_json_extended.csv  (rows=297)


WindowsPath('C:/Users/ishaa/OneDrive/Desktop/Features-main/Features-main/data/output/vehicle_alerts_decoded_json_extended.csv')